In [1]:
import sys
sys.path.append('../')

from config import zcorpus_fname, feature_manager_fname
from config import word2vec_model_fname
from config import z_mm_fname, z_mm_xwords_fname, z_mm_feature_vocab_fname

from simple_ner import get_process_memory, write_list
from simple_ner import FeatureManager, ZCorpus
from simple_ner import zcorpus_to_sparsematrix

zcorpus = ZCorpus(zcorpus_fname)
feature_manager = FeatureManager()
feature_manager.load(feature_manager_fname)

In [2]:
for i,z in enumerate(zcorpus):
    if i > 5: break
    print(z)

('아뇨', ['4428'])
('내꺼', ['1666', '1637', '30215'])
('아닌데', ['1798', '2783', '185588', '1'])
('요', ['4007', '2202'])
('네', ['8514'])
('은이', ['75', '360453'])


In [40]:
x, int2word, feature_vocab = zcorpus_to_sparsematrix(zcorpus,
                                                     feature_manager,
                                                     ner_min_feature_count=20,
                                                     pruning_per_instance=1000000,
                                                     pruning_min_featuer_count=2)

transforming zcorups to sparse matrix was done. #words= 69780 mem= 6.176 Gb
It returns (x, int2word)


In [41]:
x.shape

(69780, 443640)

In [42]:
len(int2word)

69780

In [43]:
from scipy.io import mmwrite
mmwrite(z_mm_fname, x)
write_list(z_mm_xwords_fname, int2word)
write_list(z_mm_feature_vocab_fname, feature_vocab)

In [79]:
from sklearn.preprocessing import normalize
x = normalize(x)

In [45]:
ner_seeds = '햄벅  빈대떡  팥죽  채소  인스턴트  고로케  홍어  크레페  치폴레  수제비  떡튀순  문어  콜라  삼김  짜왕  꽃게  복숭아  요구르트  쌈도  야식  한우  칼국수  해물찜  쌀밥  육개장  골뱅이  후식  아구찜  컵밥  음료수  고기  낙지  유자차  조개구이  비비큐  꼬치  케익  꼬기  청국장  갈비탕  닭발  피자  킹크랩  견과류  에스프레소  훠궈  샤브  엽떡  쫄면  아점  상추  소스  가래떡  석식  쿨피스  삼치  프링글스  카레  식혜  키위  맥주  석류  짜파게티  순대국  씨리얼  컵라면  초밥  껍데기  와플  비빔밥  치킨  음료  헛개수  뼈해장국  등갈비  갈비  두부  핫도그  라멘  잡채밥  파닭  순대  밀크티  호떡  치맥  미숫가루  떡볶이  소고기  햄버거  갈비찜  닭도  포카리  불고기  딤섬  타코야끼  김밥  탕수육  미역국  먹이  장어  갈매기살  청심환  삼겹살  광어  삼겹  수박  곱창  핫식스  풀떼기  팥빙수  마늘  쌈은  간식  주먹밥  김치찌개  닭강정  식초  월남쌈  국밥  브런치  블루베리  양꼬치  감자  홈런볼  호박죽  우유  쟁반짜장  스무디  빙수  쌈이랑  맥모닝  쌀국수  소주  과일  싸이버거  고등어  감튀  밀가루  소맥  컵누들  햇반  분식  닭갈비  떡갈비  족발  쉐이크  닭도리탕  반마리  바나나  보드카  고구마  분유  환타  쥐포  참외  뻥튀기  홍시  해장국  감자탕  살안찌  육포  소세지  양파  버블티  송편  깐풍기  파전  순두부  브리또  옥수수  츄러스  쥬스  치즈  짜장면  소금구이  쌈이  찜닭  백숙  곰국  콩국수  스시  라묜  팝콘  닭백숙  돈까스  야채  오징어  참치  연어  커리  빅맥  새우  삼계탕  토마토  닭꼬치  닭가슴살  간장게장  칙힌  갈치  물회  냉면  꼼장어  오뎅탕  돈부리  생선구이  티라미수  파스타  청하  타코  군만두  떡국  베지밀  설렁탕  아이스크림  북어국  라볶이  게장  순하리  몽쉘  쭈꾸미  추어탕  해물탕  점심  디저트  체리  샤브샤브  누룽지  꼬막  짬뽕  홍삼  샌드위치  조개찜  박카스  우동  육회  와퍼  핫바  바베큐  부대찌개  과메기  과자  수제버거  쌍화탕  밀면  꽃게탕  사과주스  바닐라라떼  샐러드  오리  비빔면  비요뜨  탕슉  목살'.split('  ')
print(len(ner_seeds))

253


In [46]:
word2int = {w:i for i,w in enumerate(int2word)}
ner_seeds_idx = {word2int[w] for w in ner_seeds if w in word2int}
len(ner_seeds_idx)

253

In [50]:
y = [1 if i in ner_seeds_idx else -1 for i in range(x.shape[0])]

In [75]:
%%time

from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(penalty='l2', C=1.0, class_weight={1:0.98, -1:0.02})
logistic.fit(x, y)
prob = logistic.predict_proba(x)

CPU times: user 11.4 s, sys: 0 ns, total: 11.4 s
Wall time: 11.4 s


In [76]:
extracted_ners = sorted(enumerate(prob[:,1]), key=lambda x:x[1], reverse=True)
extracted_ners = [(int2word[i], p) for i, p in extracted_ners]

In [80]:
extracted_ners[300:600]

[('와구와구', 0.75455136533211131),
 ('빵좀', 0.75434186826817506),
 ('컵밥', 0.75391657356372188),
 ('채소', 0.75252572845395438),
 ('이슬만', 0.75244093109711507),
 ('순대국', 0.75091133451959113),
 ('햄벅', 0.74987688593887414),
 ('소주', 0.7494036291686621),
 ('쟁반짜장', 0.74888395710599487),
 ('매운갈비찜', 0.74711607783625078),
 ('비냉', 0.74555245299038864),
 ('선지해장국', 0.74541090292463352),
 ('꿔바로우', 0.74491897331775525),
 ('만둣국', 0.74387905624217476),
 ('야채', 0.74300787210535713),
 ('부침개', 0.74281702980292363),
 ('와인', 0.73948228069103172),
 ('도시락', 0.73796337022847214),
 ('송어회', 0.73743146021596651),
 ('너구리', 0.73713601460205569),
 ('소맥', 0.73630704546827119),
 ('빠삐코', 0.73576085718909512),
 ('귤도', 0.73391901309480734),
 ('수육', 0.73380377902968141),
 ('파니니', 0.73331285018254944),
 ('김밥', 0.73242331727682541),
 ('죠떡', 0.73236629975449763),
 ('떡', 0.73092333995646364),
 ('내장탕', 0.72961539848829737),
 ('말랑카우', 0.72807270257616141),
 ('베지밀', 0.72757873752573721),
 ('생선', 0.72660026047176618),
 ('빈대떡', 0.725352